# Python code

## Imports

In [1]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

In [2]:
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

## Variables globales

In [3]:
PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.1 #in seconds
TIME_SLEEP_DATA = 0.5 # 0.2 = 5Hz frequency
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4
COMPTEUR = 6
DIST_WHEELS = 9.4

## Functions

In [ ]:
def set_course(target, x, y, alpha):
    
    dy = target[1] - y
    dx = target[0] - x
    alphatarg = np.arctan2(dy,dx)
    dalpha = alphatarg - alpha
    
    print('delta ',dx,dy,dalpha)
    
    if  dalpha > 0.05 :
        robot.set_var("motor.left.target", int(2**16)-int(MOTOR_SPEED*dalpha))
        robot.set_var("motor.right.target", int(MOTOR_SPEED*dalpha)) 
    
    elif dalpha < -0.05:
        robot.set_var("motor.left.target", int(MOTOR_SPEED*(-dalpha)))
        robot.set_var("motor.right.target", int(2**16)-int(MOTOR_SPEED*(-dalpha)))
    
    else:
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", MOTOR_SPEED) 
    return

In [ ]:
def obstacle_avoidance(obstacle_direction,sensor_measurment):
    if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
        if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
        else:
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
    else:
        if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
        elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
        elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
            robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED)
    return

In [ ]:
def go_to_goal(target_list):
    target_reached =0
    last_target_reached =0
    follow_obstacle = 0
    iteration = 0
    id_target = 1
    img = get_image()
    (robot_pos, start, goal) = get_data(img)
    x=robot_pos[0]
    y=robot_pos[1]
    alpha = robot_pos[2]
    
    while (not last_target_reached) :
        target = target_list[id_target] 
        
        if target_list[len(target_list)-1] != goal
            return 0
        
        if follow_obstacle == 1 :
            robot.set_var("motor.left.target", MOTOR_SPEED)        
            robot.set_var("motor.right.target", MOTOR_SPEED)       
        else :
            set_course(target, x, y, alpha)

        sensor_measurment = robot["prox.horizontal"]

        if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
            pass
        else:
            obstacle_direction = np.nonzero(sensor_measurment)[0]
            while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
                #robot.set_var('leds.top', 123)

                obstacle_avoidance(obstacle_direction,sensor_measurment)

                time.sleep(TIME_SLEEP_OBSTACLE)

                sensor_measurment = robot["prox.horizontal"]
                obstacle_direction = np.nonzero(sensor_measurment)[0]
                
            id_target = id_target + 1
            if id_target>len(target_list)-1:
                id_target = len(target_list)-1
            
        if (sensor_measurment[0] > 1000) or (sensor_measurment[4] > 1000) :
            follow_obstacle = 1
            iteration = 0
        else : 
            iteration = iteration + 1
            if iteration == COMPTEUR :
                follow_obstacle = 0
            
        time.sleep(TIME_SLEEP_DATA)
        
        img = get_image()
        (robot_pos, start, goal) = get_data(img)
        x=robot_pos[0]
        y=robot_pos[1]
        alpha = robot_pos[2]

        if -0.75 < x-target[0] < 0.75 and -0.75 < y-target[1] < 0.75 :
            target_reached = 1
            print('coucou')
            id_target = id_target+1
            if id_target>len(target_list)-1:
                id_target = len(target_list)-1
            
        if -1 < x-goal[0] < 1 and -1 < y-goal[1] < 1 :
            last_target_reached = 1
            robot.set_var("motor.left.target", 0)
            robot.set_var("motor.right.target", 0) 
    return 1

## Main

In [7]:
while(not(go_to_goal(path))):
    print('o0')

NameError: name 'go_to_goal' is not defined